In [1]:
data_location = 'sqlite:///../../../data/data.db'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce


import pickle

In [3]:
def vol_ohlc(df, lookback=10):
    o = df.open
    h = df.high
    l = df.low
    c = df.close
    
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    result[:lookback-1] = np.nan
    
    return result

In [4]:
ohlc = pd.read_sql('SELECT * FROM ohlc', data_location)
ohlc.shape

(11627, 9)

In [5]:
ohlc.head()

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-11-01 00:00:00,61421.37,61669.14,61239.60,61343.68,256.433869,1.575751e+07,BTC,BTC
1,2021-11-01 01:00:00,61346.17,61709.82,61171.22,61610.93,332.481185,2.044558e+07,BTC,BTC
2,2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.250720,1.935390e+07,BTC,BTC
3,2021-11-01 03:00:00,61333.17,61457.28,60050.00,60589.06,1059.931358,6.414625e+07,BTC,BTC
4,2021-11-01 04:00:00,60590.23,60655.00,59752.92,59971.89,621.419878,3.744744e+07,BTC,BTC


In [6]:
ohlc.describe()

,open,high,low,close,volume,volumeUSD
count,11627.000000,11627.000000,11627.000000,11627.000000,1.162700e+04,1.162700e+04
mean,5708.198992,5737.512791,5676.641523,5706.967946,7.782893e+05,9.847622e+06
std,16518.161143,16599.532113,16430.972527,16514.731530,2.057505e+06,1.769057e+07
min,0.999900,1.000000,0.995100,0.999900,6.713000e+00,1.960784e+03
25%,4.555900,4.611000,4.496050,4.554350,2.565695e+03,9.664755e+05
50%,92.590000,93.710000,91.000000,92.600000,4.624230e+04,3.420994e+06
75%,307.924500,309.700000,305.501000,307.796500,1.768436e+05,1.068309e+07
max,68638.470000,69000.000000,68456.500000,68639.630000,3.978895e+07,3.988035e+08


To predict SOL return, I need to feature engineer the most relevant elements. 

In [7]:
tokens = ohlc.token.unique()

In [8]:
def df_merge(left, right):
    return pd.merge(left, right, on='ts', how='inner')

X = reduce(df_merge, [
    (lambda df: 
    (
        df
        .assign(
            vol=vol_ohlc(df).fillna(0),
            ret=df.close.pct_change()
        )[['ts', 'vol', 'ret']]
        .rename(columns={
            col: f'{col}_{token}' for col in ['ts', 'vol', 'ret'] if col != 'ts'
        })
    ))(ohlc[ohlc.token == token])
    for token in tokens
]).set_index('ts')

In [9]:
X.tail()

,vol_BTC,ret_BTC,vol_ETH,ret_ETH,vol_USDT,ret_USDT,vol_SOL,ret_SOL,vol_ADA,ret_ADA,...,vol_AVAX,ret_AVAX,vol_ATOM,ret_ATOM,vol_CRV,ret_CRV,vol_AAVE,ret_AAVE,vol_COMP,ret_COMP
ts,,,,,,,,,,,,,,,,,,,,,
2021-12-14 20:00:00,0.136358,0.004810,0.158369,0.005961,0.002463,0.0001,0.216740,0.002798,0.225286,0.014437,...,0.239258,0.008227,0.254118,-0.000471,0.255464,0.003049,0.207758,0.009387,0.284268,0.006813
2021-12-14 21:00:00,0.142237,0.019797,0.170096,0.016737,0.002652,0.0000,0.218492,0.025892,0.224116,0.012141,...,0.241603,0.019207,0.254350,0.019303,0.263456,0.021277,0.222014,0.022490,0.281497,0.017473
2021-12-14 22:00:00,0.151148,0.010414,0.172081,0.004623,0.002684,0.0000,0.246122,0.015624,0.232362,0.009295,...,0.268875,0.026851,0.253472,0.012933,0.270895,0.023810,0.220598,0.006242,0.275083,0.009868
2021-12-14 23:00:00,0.149424,-0.000302,0.170257,-0.003195,0.002823,0.0001,0.238235,-0.010027,0.231115,-0.000157,...,0.301778,0.032506,0.245708,-0.009576,0.268758,-0.011628,0.219474,-0.003013,0.271229,-0.000864
2021-12-15 00:00:00,0.143079,-0.001448,0.159883,-0.000641,0.002816,0.0000,0.228867,0.002517,0.220068,-0.007715,...,0.296444,0.005576,0.234522,-0.003223,0.242546,-0.002941,0.213360,0.001659,0.246067,-0.001027


Add more feature information from SOL (open, high, low, close, volume, volumeUSD) for its return prediction. Meanwhile, I will use some of technical analysis indicators for prediction purpose.

In [10]:
SOL_df = ohlc[ohlc.token == 'SOL'].set_index('ts')

In [11]:
SOL_df

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-11-01 00:00:00,202.194,203.225,200.701,202.673,38155.628,7.720360e+06,SOL,SOL
2021-11-01 01:00:00,202.673,209.736,202.087,208.218,157689.578,3.260716e+07,SOL,SOL
2021-11-01 02:00:00,208.349,210.203,205.693,206.161,112186.097,2.337273e+07,SOL,SOL
2021-11-01 03:00:00,206.161,206.788,199.212,201.689,141171.549,2.858246e+07,SOL,SOL
2021-11-01 04:00:00,201.628,203.995,200.562,201.076,63368.043,1.280768e+07,SOL,SOL
...,...,...,...,...,...,...,...,...
2021-12-14 20:00:00,153.660,154.510,152.890,154.100,44436.935,6.826619e+06,SOL,SOL
2021-12-14 21:00:00,154.110,158.090,154.020,158.090,72284.661,1.127975e+07,SOL,SOL
2021-12-14 22:00:00,158.090,163.360,157.360,160.560,104942.890,1.686540e+07,SOL,SOL


In [12]:
!pip install pandas_ta

In [13]:
import pandas_ta as ta

In [14]:
SOL_df.ta.sma(length=10, append=True)

ts
2021-11-01 00:00:00        NaN
2021-11-01 01:00:00        NaN
2021-11-01 02:00:00        NaN
2021-11-01 03:00:00        NaN
2021-11-01 04:00:00        NaN
                        ...   
2021-12-14 20:00:00    156.632
2021-12-14 21:00:00    156.874
2021-12-14 22:00:00    157.112
2021-12-14 23:00:00    157.072
2021-12-15 00:00:00    157.204
Name: SMA_10, Length: 1057, dtype: float64

In [15]:
SOL_df.ta.macd(append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
ts,,,
2021-11-01 00:00:00,NaN,NaN,NaN
2021-11-01 01:00:00,NaN,NaN,NaN
2021-11-01 02:00:00,NaN,NaN,NaN
2021-11-01 03:00:00,NaN,NaN,NaN
2021-11-01 04:00:00,NaN,NaN,NaN
...,...,...,...
2021-12-14 20:00:00,-1.266631,0.172408,-1.439038
2021-12-14 21:00:00,-0.980853,0.366548,-1.347401
2021-12-14 22:00:00,-0.548738,0.638931,-1.187669


In [16]:
SOL_df.ta.rsi(append=True)

ts
2021-11-01 00:00:00          NaN
2021-11-01 01:00:00          NaN
2021-11-01 02:00:00          NaN
2021-11-01 03:00:00          NaN
2021-11-01 04:00:00          NaN
                         ...    
2021-12-14 20:00:00    42.919353
2021-12-14 21:00:00    52.038937
2021-12-14 22:00:00    56.655598
2021-12-14 23:00:00    53.069689
2021-12-15 00:00:00    53.851201
Name: RSI_14, Length: 1057, dtype: float64

In [17]:
SOL_df.shape

(1057, 13)

In [18]:
X = X.join(SOL_df)

In [19]:
X.head(50)

,vol_BTC,ret_BTC,vol_ETH,ret_ETH,vol_USDT,ret_USDT,vol_SOL,ret_SOL,vol_ADA,ret_ADA,...,close,volume,volumeUSD,token,chain,SMA_10,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14
ts,,,,,,,,,,,,,,,,,,,,,
2021-11-01 00:00:00,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,...,202.673,38155.628,7.720360e+06,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 01:00:00,0.000000,0.004357,0.000000,0.006874,0.000000,0.0000,0.000000,0.027359,0.000000,0.003203,...,208.218,157689.578,3.260716e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 02:00:00,0.000000,-0.004508,0.000000,-0.005322,0.000000,-0.0002,0.000000,-0.009879,0.000000,-0.008667,...,206.161,112186.097,2.337273e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 03:00:00,0.000000,-0.012132,0.000000,-0.013126,0.000000,0.0001,0.000000,-0.021692,0.000000,-0.007618,...,201.689,141171.549,2.858246e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 04:00:00,0.000000,-0.010186,0.000000,-0.010679,0.000000,0.0000,0.000000,-0.003039,0.000000,-0.006903,...,201.076,63368.043,1.280768e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 05:00:00,0.000000,0.009699,0.000000,0.008355,0.000000,0.0001,0.000000,0.007863,0.000000,0.001349,...,202.657,85632.361,1.721096e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 06:00:00,0.000000,0.001454,0.000000,-0.000376,0.000000,-0.0001,0.000000,0.003938,0.000000,0.002487,...,203.455,46693.992,9.522210e+06,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 07:00:00,0.000000,0.002006,0.000000,0.003566,0.000000,-0.0001,0.000000,0.020442,0.000000,0.004289,...,207.614,61474.116,1.261825e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN
2021-11-01 08:00:00,0.000000,0.023630,0.000000,0.014728,0.000000,0.0000,0.000000,0.011189,0.000000,0.017598,...,209.937,102762.420,2.147337e+07,SOL,SOL,NaN,NaN,NaN,NaN,NaN


Use this dataframe as the input data

In [20]:
y = X.ret_SOL.shift(-1)[:-1]
X = X[:-1]

In [23]:
X = X.drop(columns=['token', 'chain'])

In [26]:
pd.isnull(X).sum()

vol_BTC           0
ret_BTC           1
vol_ETH           0
ret_ETH           1
vol_USDT          0
ret_USDT          1
vol_SOL           0
ret_SOL           1
vol_ADA           0
ret_ADA           1
vol_DOT           0
ret_DOT           1
vol_AVAX          0
ret_AVAX          1
vol_ATOM          0
ret_ATOM          1
vol_CRV           0
ret_CRV           1
vol_AAVE          0
ret_AAVE          1
vol_COMP          0
ret_COMP          1
open              0
high              0
low               0
close             0
volume            0
volumeUSD         0
SMA_10            9
MACD_12_26_9     25
MACDh_12_26_9    33
MACDs_12_26_9    33
RSI_14           14
dtype: int64

I decided to drop the first row, as returns are not availabe. Meanwhile, for technical analysis indicators, I will need to impute their values due to high N.A. 

In [27]:
X=X[1:]
y=y[1:]

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.model_selection import learning_curve

In [29]:
X = X.bfill(axis ='rows') #backfill the data as they are correlated along the time axis

In [30]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]


In [31]:
def evaluate_model(model, X, y, test_size=0.2):
    cv =  TimeSeriesSplit(n_splits=int(y.shape[0] * test_size), test_size=1)
    scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)
    
    return np.mean(cross_validate(model, X, y, cv=cv, scoring=scorer, n_jobs=-1)['test_score'])

In [32]:
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=15)),
    ('model', Ridge(alpha=1.0))
])

evaluate_model(pipeline, X, y)

-0.008791592816754252

In [33]:
pipeline = Pipeline([
    ('model', DecisionTreeRegressor(random_state=0))
])

#scaling might not be a good thing for this model

evaluate_model(pipeline, X, y)

-0.012431111845871479

In [34]:
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('model', Ridge(alpha=1.0))
])

evaluate_model(pipeline, X, y)

-0.008733135956658982

In [35]:
X.columns

Index(['vol_BTC', 'ret_BTC', 'vol_ETH', 'ret_ETH', 'vol_USDT', 'ret_USDT',
       'vol_SOL', 'ret_SOL', 'vol_ADA', 'ret_ADA', 'vol_DOT', 'ret_DOT',
       'vol_AVAX', 'ret_AVAX', 'vol_ATOM', 'ret_ATOM', 'vol_CRV', 'ret_CRV',
       'vol_AAVE', 'ret_AAVE', 'vol_COMP', 'ret_COMP', 'open', 'high', 'low',
       'close', 'volume', 'volumeUSD', 'SMA_10', 'MACD_12_26_9',
       'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_14'],
      dtype='object')

In [57]:
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('model', Ridge(alpha=1.0))
])

evaluate_model(pipeline, X, y)

-0.008733135956658982

In [41]:
from sklearn import svm

In [42]:
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('model', svm.SVR())
])

evaluate_model(pipeline, X, y)

-0.008546201345938906

In [43]:
pipeline = Pipeline([
    ('feature_selector', FeatureSelector(['ret_SOL', 'SMA_10', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_14'])),
    ('scale', StandardScaler()),
    ('model', svm.SVR())
])

evaluate_model(pipeline, X, y)

-0.008546201345938906

For svm, it looks like the featureselector doesn't change the score

Try to optimize the hyperparameter

In [46]:
svm.SVC().get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [52]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('model', svm.SVR())
])

evaluate_model(pipeline, X, y)

test_size = 0.2
cv = TimeSeriesSplit(n_splits=int(y.shape[0] * test_size), test_size=1)
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

search = GridSearchCV(pipeline, {
    'model__C': [0.1, 0.5, 1, 2],
    'model__gamma': [1, 0.1, 0.01, 0.001, 0.0001]
}, scoring=scorer, refit=True, cv=cv, n_jobs=-1)
#here you can search the hyperparameter of the pipeline; 
#the name of the hyperparameter is 'name of the pipeline item'__'hyperparameter_name' 

search.fit(X, y)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=211, test_size=1),
             estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('model', SVR())]),
             n_jobs=-1,
             param_grid={'model__C': [0.1, 0.5, 1, 2],
                         'model__gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))

In [53]:
search.best_params_

{'model__C': 0.1, 'model__gamma': 1}

In [54]:
best_model = search.best_estimator_

In [55]:
evaluate_model(best_model, X, y)

-0.008546201345938906

In [56]:
pipeline = Pipeline([
    ('feature_selector', FeatureSelector(['vol_BTC', 'ret_BTC', 'vol_ETH', 'ret_ETH', 'vol_USDT', 'ret_USDT',
       'vol_SOL', 'ret_SOL', 'vol_ADA', 'ret_ADA', 'vol_DOT', 'ret_DOT',
       'vol_AVAX', 'ret_AVAX', 'vol_ATOM', 'ret_ATOM', 'vol_CRV', 'ret_CRV',
       'vol_AAVE', 'ret_AAVE', 'vol_COMP', 'ret_COMP'])),
    ('scale', StandardScaler()),
    ('model', svm.SVR())
])

evaluate_model(pipeline, X, y)

C:\Users\15103\Conda\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


-0.008546201345938906